<a href="https://colab.research.google.com/github/msri200/Medium-Frequency-Trading-Portfolio/blob/main/medfreq_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install quandl -q

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import quandl
%matplotlib inline

In [ ]:
#import quandl

#quandl.ApiConfig.api_key = "HWQp1f4j8r5CNxWWv2M3"

In [ ]:
import yfinance as yf

zn = yf.download("ZN=F", start="1985-01-01", end="2020-10-01")
zf = yf.download("ZF=F", start="1985-01-01", end="2020-10-01")

/tmp/ipython-input-484264592.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  zn = yf.download("ZN=F", start="1985-01-01", end="2020-10-01")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-484264592.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  zf = yf.download("ZF=F", start="1985-01-01", end="2020-10-01")
[*********************100%***********************]  1 of 1 completed


In [ ]:
print(zn.head())
print(zf.head())

Price            Close        High         Low        Open Volume
Ticker            ZN=F        ZN=F        ZN=F        ZN=F   ZN=F
Date                                                             
2000-09-21   99.953133   99.953133   99.625000   99.656250  41462
2000-09-22   99.828133   99.875000   99.828133   99.859383  35044
2000-09-25   99.937500   99.968750   99.937500   99.937500  35198
2000-09-26  100.187500  100.234398  100.140602  100.171898  52999
2000-09-27  100.125000  100.140602  100.093803  100.109398  39859
Price            Close        High         Low        Open Volume
Ticker            ZF=F        ZF=F        ZF=F        ZF=F   ZF=F
Date                                                             
2000-09-21  100.421898  100.468803  100.171898  100.187500  10742
2000-09-22  100.281303  100.328102  100.281303  100.296898   6092
2000-09-25  100.296898  100.328102  100.296898  100.296898   6359
2000-09-26  100.484398  100.500000  100.421898  100.437500   8689
2000-09-27

In [ ]:
# Flatten and lower-case to basically turn this from multi-dimensional to 1D array
zf.columns = [c[0].lower() for c in zf.columns]
zn.columns = [c[0].lower() for c in zn.columns]

In [ ]:
zf.info() #quick summary of non-null values
zf.isna().sum() #Nan count per column
zn.info()
zn.isna().sum()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5038 entries, 2000-09-21 to 2020-09-30
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   close   5038 non-null   float64
 1   high    5038 non-null   float64
 2   low     5038 non-null   float64
 3   open    5038 non-null   float64
 4   volume  5038 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 236.2 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5026 entries, 2000-09-21 to 2020-09-30
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   close   5026 non-null   float64
 1   high    5026 non-null   float64
 2   low     5026 non-null   float64
 3   open    5026 non-null   float64
 4   volume  5026 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 235.6 KB


,0
close,0
high,0
low,0
open,0
volume,0


In [ ]:
zf = zf.rename(columns={
    'o':'open',
    'c':'close',
    'h':'high',
    'l':'low',
    'v':'volume',
})[['open','close','high','low','volume']]

zn = zn.rename(columns={
    'o':'open',
    'c':'close',
    'h':'high',
    'l':'low',
    'v':'volume',
})[['open','close','high','low','volume']]

In [ ]:
zfcopy = zf.copy()
zncopy = zn.copy()

In [ ]:
zfcopy = zfcopy.assign(midprice = zfcopy['open'] + zfcopy['close'])/2
zncopy = zncopy.assign(midprice = zncopy['open'] + zncopy['close'])/2

In [ ]:
print(zncopy.head())
print(zfcopy.head())

                 open      close       high        low   volume    midprice
Date                                                                       
2000-09-21  49.828125  49.976566  49.976566  49.812500  20731.0   99.804691
2000-09-22  49.929691  49.914066  49.937500  49.914066  17522.0   99.843758
2000-09-25  49.968750  49.968750  49.984375  49.968750  17599.0   99.937500
2000-09-26  50.085949  50.093750  50.117199  50.070301  26499.5  100.179699
2000-09-27  50.054699  50.062500  50.070301  50.046902  19929.5  100.117199
                 open      close       high        low  volume    midprice
Date                                                                      
2000-09-21  50.093750  50.210949  50.234402  50.085949  5371.0  100.304699
2000-09-22  50.148449  50.140652  50.164051  50.140652  3046.0  100.289101
2000-09-25  50.148449  50.148449  50.164051  50.148449  3179.5  100.296898
2000-09-26  50.218750  50.242199  50.250000  50.210949  4344.5  100.460949
2000-09-27  50.234

In [ ]:
pip install rateslib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.0/739.0 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for rateslib: filename=rateslib-2.1.1-cp310-abi3-linux_x86_64.whl size=1790039 sha256=ce163304c2ac2abc32c9e899dd9512480b461c727de874aa21b307733734d0c6
  Stored in directory: /root/.cache/pip/wheels/2d/68/9b/0b0fa29aa8215201ffe9cac962ae3522027ed6a99b9dd9de3a
Successfully built rateslib


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer

In [ ]:
#we split the data firstly into train and test where test contains validation as well
#then we split test and validation
zf_train, zf_temp = train_test_split(zfcopy, test_size=0.2,train_size= 0.8, shuffle=False)
zn_train, zn_temp = train_test_split(zncopy, test_size=0.2, train_size= 0.8, shuffle=False)
zf_validation, zf_test = train_test_split(zf_temp, test_size=0.5, shuffle=False)
zn_validation, zn_test = train_test_split(zn_temp,test_size = 0.5,shuffle= False )
#we DO NOT do a k-fold validation as we are only interested in how a
#historical model performs on new data

In [ ]:
#calculate our raw features
zncopy['Weekly_SMA'] = zncopy['midprice'].rolling(window=7).mean()
zncopy['Biweekly_SMA'] = zncopy['midprice'].rolling(window=14).mean()
zfcopy['Weekly_SMA'] = zfcopy['midprice'].rolling(window=7).mean()
zfcopy['Biweekly_SMA'] = zfcopy['midprice'].rolling(window=14).mean()
zfcopy['Daily_Trend_Weekly_MA']= zfcopy['midprice'] - zfcopy['Weekly_SMA']
zncopy['Daily_Trend_Weekly_MA']= zncopy['midprice'] - zncopy['Weekly_SMA']
zfcopy['Daily_Trend_Biweekly_MA']= zfcopy['midprice'] - zfcopy['Biweekly_SMA']
zncopy['Daily_Trend_Biweekly_MA']= zncopy['midprice'] - zncopy['Biweekly_SMA']

print(zncopy.head(20))
print(zfcopy.head(20))

                 open      close       high        low   volume    midprice  \
Date                                                                          
2000-09-21  49.828125  49.976566  49.976566  49.812500  20731.0   99.804691   
2000-09-22  49.929691  49.914066  49.937500  49.914066  17522.0   99.843758   
2000-09-25  49.968750  49.968750  49.984375  49.968750  17599.0   99.937500   
2000-09-26  50.085949  50.093750  50.117199  50.070301  26499.5  100.179699   
2000-09-27  50.054699  50.062500  50.070301  50.046902  19929.5  100.117199   
2000-09-28  50.070301  50.101551  50.109402  50.046902  19035.5  100.171852   
2000-09-29  50.132801  50.109402  50.132801  50.101551  11549.5  100.242203   
2000-10-02  50.093750  50.101551  50.101551  50.085949  15387.5  100.195301   
2000-10-03  49.968750  49.968750  50.031250  49.953125  20870.5   99.937500   
2000-10-04  49.875000  49.898441  49.898441  49.875000  19361.5   99.773441   
2000-10-05  49.992191  50.007801  50.007801  49.9843

In [ ]:
# remove rows with ANY NaN in either dataframe
zncopy = zncopy.dropna().copy()
zfcopy = zfcopy.dropna().copy()
print(zncopy.head(20))
print(zfcopy.head(20))

                 open      close       high        low   volume    midprice  \
Date                                                                          
2000-10-10  50.218750  50.281250  50.289051  50.195301  35246.0  100.500000   
2000-10-11  50.320301  50.296902  50.335949  50.289051  31509.5  100.617203   
2000-10-12  50.265652  50.507801  50.578152  50.195301  15999.0  100.773453   
2000-10-13  50.500000  50.453152  50.601551  50.390652  15738.0  100.953152   
2000-10-16  50.390652  50.421902  50.460949  50.328152  25726.0  100.812553   
2000-10-17  50.429699  50.625000  50.664051  50.359402  33292.0  101.054699   
2000-10-18  50.671902  50.625000  51.031250  50.554699  23250.0  101.296902   
2000-10-19  50.609402  50.687500  50.781250  50.500000  13649.0  101.296902   
2000-10-20  50.648449  50.773449  50.812500  50.632801  19822.5  101.421898   
2000-10-23  50.781250  50.929699  50.953152  50.773449  21856.5  101.710949   
2000-10-24  50.914051  50.828152  50.945301  50.6953

In [ ]:
#A. PORTFOLIO CONSTRUCTION
from sklearn.preprocessing import MinMaxScaler      # or StandardScaler
from sklearn.decomposition import PCA

#Build aligned 2-col DataFrames (dates must match)
train_df = pd.concat([zf_train['midprice'], zn_train['midprice']], axis=1, join='inner')
train_df.columns = ['zf_mid', 'zn_mid']

val_df   = pd.concat([zf_validation['midprice'], zn_validation['midprice']], axis=1, join='inner')
val_df.columns = ['zf_mid', 'zn_mid']

test_df  = pd.concat([zf_test['midprice'], zn_test['midprice']], axis=1, join='inner')
test_df.columns = ['zf_mid', 'zn_mid']

#Fit scaler on TRAIN only (preserves relative scale)
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_df)   # shape (n_train, 2)
val_scaled   = scaler.transform(val_df)        # do NOT fit again
test_scaled  = scaler.transform(test_df)

# Debug check
print("train_scaled.shape:", train_scaled.shape)
print("val_scaled.shape:", val_scaled.shape)

# Fit PCA on TRAIN only; use 2 components so we have PC2 available
pca = PCA(n_components=2)
pca.fit(train_scaled)

# Debug check
print("pca.components_.shape:", pca.components_.shape)  # should be (2, 2)

#Get PC scores (projections). Use transform() — clean and correct.
train_scores = pca.transform(train_scaled)   # shape (n_train, 2)
val_scores   = pca.transform(val_scaled)
test_scores  = pca.transform(test_scaled)

# PC2 is the second column (index 1)
train_portfolio = pd.Series(train_scores[:, 1], index=train_df.index, name='train_portfolio_price')
val_portfolio   = pd.Series(val_scores[:, 1],   index=val_df.index, name='val_portfolio_price')
test_portfolio  = pd.Series(test_scores[:, 1],  index=test_df.index, name='test_portfolio_price')

# Quick sanity prints
print(train_portfolio.head())
print(val_portfolio.head())
print(test_portfolio.head())


train_scaled.shape: (4019, 2)
val_scaled.shape: (502, 2)
pca.components_.shape: (2, 2)
Date
2000-09-21    0.019097
2000-09-22    0.020329
2000-09-25    0.022019
2000-09-26    0.022357
2000-09-27    0.020865
Name: train_portfolio_price, dtype: float64
Date
2016-09-28    0.065026
2016-09-29    0.065146
2016-09-30    0.062780
2016-10-03    0.064073
2016-10-04    0.062126
Name: val_portfolio_price, dtype: float64
Date
2018-10-02    0.063493
2018-10-03    0.062142
2018-10-04    0.059082
2018-10-05    0.057532
2018-10-08    0.055584
Name: test_portfolio_price, dtype: float64


In [ ]:
    pip install scikit-learn

In [ ]:
    pip install tensorflow

In [ ]:
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [ ]:
#initialize the rbm model
#rbm =BernoulliRBM(n_components= 256, learning_rate= 0.001, n_iter= 100, verbose= 1, random_state= 0)

In [ ]:
training_portfolio_set   = torch.tensor(train_portfolio.values, dtype=torch.float32).unsqueeze(1)
validation_portfolio_set = torch.tensor(val_portfolio.values,  dtype=torch.float32).unsqueeze(1)
test_portfolio_set       = torch.tensor(test_portfolio.values,  dtype=torch.float32).unsqueeze(1)

In [ ]:
print(training_portfolio_set.shape)

torch.Size([4019, 1])


In [ ]:
import torch

class RBM:
    def __init__(self, n_visible, n_hidden):
        # W is [n_visible, n_hidden] so v @ W -> [batch, n_hidden]
        self.W = torch.randn(n_visible, n_hidden) * 0.1
        self.v_bias = torch.zeros(1, n_visible)
        self.h_bias = torch.zeros(1, n_hidden)

    def sample_h_givenv(self, v):
        h_prob = torch.sigmoid(v @ self.W + self.h_bias)      # no transpose
        h_sample = torch.bernoulli(h_prob)
        return h_prob, h_sample

    def sample_v_givenh(self, h):
        v_prob = torch.sigmoid(h @ self.W.T + self.v_bias)    # transpose here only
        v_sample = torch.bernoulli(v_prob)
        return v_prob, v_sample

    def train(self, v0, vk, ph0, phk, lr=0.01):
        batch = v0.size(0)
        # v0.T @ ph0 -> [n_visible, n_hidden] matches self.W
        self.W += lr * ((v0.T @ ph0 - vk.T @ phk) / batch)
        self.h_bias += lr * (ph0 - phk).sum(0, keepdim=True) / batch
        self.v_bias += lr * (v0 - vk).sum(0, keepdim=True) / batch


# - usage -
v = training_portfolio_set
if v.dim() == 1:            # ensure [N, D]
    v = v.unsqueeze(1)

nv = v.shape[1]
nh = 15
rbm1 = RBM(nv, nh)

ph, h_sample = rbm1.sample_h_givenv(v)
nv2 = ph.shape[1]
nh2 = 20
rbm2 = RBM(nv2, nh2)

# training the rbms
batch_size = 100
epochs = 100
rbmtrain1 = RBM(n_visible=v.shape[1], n_hidden=15)

for epoch in range(epochs):
    for i in range(0, v.size(0), batch_size):
        v0 = v[i:i+batch_size]
        h0_prob = torch.sigmoid(v0 @ rbmtrain1.W + rbmtrain1.h_bias)

        v1_prob = torch.sigmoid(h0_prob @ rbmtrain1.W.T + rbmtrain1.v_bias)
        v1 = torch.bernoulli(v1_prob)

        h1_prob = torch.sigmoid(v1 @ rbmtrain1.W + rbmtrain1.h_bias)

        rbmtrain1.train(v0, v1, h0_prob, h1_prob)
